In [45]:
from langchain_community.utilities import SQLDatabase
from langchain_anthropic import ChatAnthropic
from langchain.chains.sql_database.prompt import SQLITE_PROMPT
from langchain.chains.sql_database.query import create_sql_query_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")
db = SQLDatabase.from_uri("sqlite:///db/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [88]:
question = "幫我找出發票表格中哪個國家的消費金額最高?"

In [89]:
sql_prompt=PromptTemplate.from_template('''
You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Question: {input}

Please only provide the SQL query in the following format:
SQL>>> <query here>                                   
''')
def sql_parser(response:str):
    return response.split("SQL>>>")[1].strip()

chain = create_sql_query_chain(llm, db,prompt=sql_prompt) | sql_parser
response = chain.invoke({"question":question})
response

'SELECT "BillingCountry", SUM("Total") as "TotalAmount"\nFROM "Invoice"\nGROUP BY "BillingCountry"\nORDER BY "TotalAmount" DESC\nLIMIT 1;'

In [90]:
print(response)

SELECT "BillingCountry", SUM("Total") as "TotalAmount"
FROM "Invoice"
GROUP BY "BillingCountry"
ORDER BY "TotalAmount" DESC
LIMIT 1;


In [91]:
db.run(response)

"[('USA', 523.06)]"

In [92]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db,prompt=sql_prompt) | sql_parser
chain = write_query | execute_query
chain.invoke({"question": question})

"[('USA', 523.06)]"

In [93]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke({"question": question}))

根據查詢結果，在發票表格中消費金額最高的國家是美國(USA)，總消費金額為523.06元。
